In [47]:
import pandas as pd
import numpy as np


In [49]:
users = pd.read_csv("C:/Users/Abhi/Desktop/Power_BI_Data_Analysis_Project/OTT-Streaming-Analytics/data/raw/users_raw.csv")
content = pd.read_csv("C:/Users/Abhi/Desktop/Power_BI_Data_Analysis_Project/OTT-Streaming-Analytics/data/raw/content_raw.csv")
viewing = pd.read_csv("C:/Users/Abhi/Desktop/Power_BI_Data_Analysis_Project/OTT-Streaming-Analytics/data/raw/viewing_activity_raw.csv")


In [51]:
users["signup_date"] = pd.to_datetime(users["signup_date"])

users = users.drop_duplicates(subset="user_id")


In [53]:
users.head()

,user_id,signup_date,country,device
0,1,2024-02-01,Canada,Mobile
1,2,2024-07-18,UK,Mobile
2,3,2023-11-11,UK,TV
3,4,2022-04-12,UK,Tablet
4,5,2021-03-29,USA,Tablet


In [55]:
content = content.drop_duplicates(subset="content_id")

content = content[content["duration_min"] > 0]


In [57]:
content.head()

,content_id,title,genre,duration_min,release_year
0,1,Content_1,Drama,120,2012
1,2,Content_2,Action,134,2021
2,3,Content_3,Thriller,143,2020
3,4,Content_4,Romance,69,2003
4,5,Content_5,Documentary,88,2023


In [59]:
viewing["date"] = pd.to_datetime(viewing["date"])


In [64]:
# Remove invalid watch time
viewing = viewing[viewing["watch_minutes"] > 0]

In [66]:
# Merge to validate duration
viewing = viewing.merge(
    content[["content_id", "duration_min"]],
    on="content_id",
    how="left"
)

In [70]:
# Cap watch time to content duration
viewing["watch_minutes"] = np.where(
    viewing["watch_minutes"] > viewing["duration_min"],
    viewing["duration_min"],
    viewing["watch_minutes"]
)

In [72]:
# Completion flag
viewing["completed_flag"] = np.where(
    viewing["watch_minutes"] >= viewing["duration_min"] * 0.9,
    1,
    0
)

In [74]:
# Drop helper column
viewing = viewing.drop(columns=["duration_min"])

In [76]:
# Remove duplicates
viewing = viewing.drop_duplicates(
    subset=["user_id", "content_id", "date"]
)

In [78]:
viewing.head()

,date,user_id,content_id,watch_minutes,completed_flag
0,2024-06-06,1,119,74,0
1,2025-01-22,1,262,67,1
2,2024-05-13,1,37,18,0
3,2024-03-08,1,8,107,0
4,2024-12-29,1,296,111,0


In [80]:
print("Users:", users.shape)
print("Content:", content.shape)
print("Viewing:", viewing.shape)

Users: (5000, 4)
Content: (300, 5)
Viewing: (109382, 5)


In [82]:
print("Completion rate:",
      viewing["completed_flag"].mean())

Completion rate: 0.16515514435647546


In [84]:
users.to_csv("C:/Users/Abhi/Desktop/Power_BI_Data_Analysis_Project/OTT-Streaming-Analytics/data/processed/clean/users_clean.csv", index=False)
content.to_csv("C:/Users/Abhi/Desktop/Power_BI_Data_Analysis_Project/OTT-Streaming-Analytics/data/processed/clean/content_clean.csv", index=False)
viewing.to_csv("C:/Users/Abhi/Desktop/Power_BI_Data_Analysis_Project/OTT-Streaming-Analytics/data/processed/clean/viewing_activity_clean.csv", index=False)